In [ ]:
from google.colab import drive
drive.mount('/content/drive')   # Accept the prompt

# Path to your WESAD folder in Drive (based on your screenshot)
DATA_ROOT = "/content/drive/MyDrive/WESAD"

import os
import pickle
import numpy as np
import pandas as pd
from scipy.signal import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


MessageError: Error: credential propagation was unsuccessful

In [ ]:
subjects = sorted([d for d in os.listdir(DATA_ROOT) if d.startswith("S")])
print("Subjects:", subjects)


Subjects: ['S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']


In [ ]:
def load_subject(subject_id):
    """
    subject_id: e.g. 'S2', 'S3'
    returns: dict loaded from Sx.pkl
    """
    pkl_path = os.path.join(DATA_ROOT, subject_id, f"{subject_id}.pkl")
    with open(pkl_path, "rb") as f:
        data = pickle.load(f, encoding="latin1")
    return data

# Test on one subject
sub_id = "S2"
data_s2 = load_subject(sub_id)

print("Top-level keys:", data_s2.keys())
print("Signal keys:", data_s2["signal"].keys())
print("Wrist channels:", data_s2["signal"]["wrist"].keys())
print("Label shape:", np.array(data_s2["label"]).shape)
print("Unique labels:", np.unique(data_s2["label"]))


Top-level keys: dict_keys(['signal', 'label', 'subject'])
Signal keys: dict_keys(['chest', 'wrist'])
Wrist channels: dict_keys(['ACC', 'BVP', 'EDA', 'TEMP'])
Label shape: (4255300,)
Unique labels: [0 1 2 3 4 6 7]


In [ ]:
# Sampling rate of wrist data in WESAD (from README): 32 Hz
FS_WRIST = 32

# Window settings (10s window, 5s step)
WINDOW_SEC = 10
STEP_SEC = 5
WINDOW = WINDOW_SEC * FS_WRIST
STEP = STEP_SEC * FS_WRIST

def map_label_multiclass_to_binary(lbl):
    """
    Map original WESAD labels to binary.

    Original (typical):
      0 = undefined / transition
      1 = baseline
      2 = stress (TSST)
      3 = amusement
      4 = meditation

    We'll treat only label 2 as 'stress' (1) and 1/3/4 as 'non-stress' (0).
    0 and anything else => ignore.
    """
    if lbl == 2:
        return 1
    elif lbl in [1, 3, 4]:
        return 0
    else:
        return -1  # ignore


In [ ]:
def get_wrist_signals_and_labels(sub_data):
    wrist = sub_data["signal"]["wrist"]
    eda = np.array(wrist["EDA"])
    bvp = np.array(wrist["BVP"])
    acc = np.array(wrist["ACC"])  # shape (N,3)
    temp = np.array(wrist["TEMP"])
    labels_full = np.array(sub_data["label"])

    # Resample labels to match wrist length
    labels_resampled = resample(labels_full, len(eda))
    labels_resampled = np.rint(labels_resampled).astype(int)

    return eda, bvp, acc, temp, labels_resampled

def extract_window_features(eda_w, bvp_w, acc_w, temp_w):
    """
    eda_w, bvp_w, temp_w: 1D arrays (window)
    acc_w: 2D array (window, 3)
    returns: dict of features
    """
    feats = {}

    # EDA
    feats["eda_mean"] = float(np.mean(eda_w))
    feats["eda_std"] = float(np.std(eda_w))
    feats["eda_min"] = float(np.min(eda_w))
    feats["eda_max"] = float(np.max(eda_w))

    # BVP
    feats["bvp_mean"] = float(np.mean(bvp_w))
    feats["bvp_std"] = float(np.std(bvp_w))

    # TEMP
    feats["temp_mean"] = float(np.mean(temp_w))
    feats["temp_std"] = float(np.std(temp_w))

    # ACC magnitude
    mag = np.linalg.norm(acc_w, axis=1)  # sqrt(x^2 + y^2 + z^2)
    feats["acc_mag_mean"] = float(np.mean(mag))
    feats["acc_mag_std"] = float(np.std(mag))

    return feats

def build_subject_windows(subject_id):
    """
    Build a list of {features..., label, subject} for one subject.
    """
    sub_data = load_subject(subject_id)
    eda, bvp, acc, temp, labels = get_wrist_signals_and_labels(sub_data)

    n = len(eda)
    rows = []

    for start in range(0, n - WINDOW, STEP):
        end = start + WINDOW

        lab_window = labels[start:end]
        # majority label in this window
        vals, counts = np.unique(lab_window, return_counts=True)
        maj_label = vals[np.argmax(counts)]
        bin_label = map_label_multiclass_to_binary(maj_label)

        # skip windows with undefined labels
        if bin_label == -1:
            continue

        feats = extract_window_features(
            eda[start:end],
            bvp[start:end],
            acc[start:end],
            temp[start:end],
        )
        feats["label"] = bin_label
        feats["subject"] = subject_id
        rows.append(feats)

    return rows


In [ ]:
rows_s2 = build_subject_windows("S2")
print("Number of windows from S2:", len(rows_s2))
print("First row keys:", rows_s2[0].keys())
print("Label distribution in S2:", pd.Series([r["label"] for r in rows_s2]).value_counts())


Number of windows from S2: 73
First row keys: dict_keys(['eda_mean', 'eda_std', 'eda_min', 'eda_max', 'bvp_mean', 'bvp_std', 'temp_mean', 'temp_std', 'acc_mag_mean', 'acc_mag_std', 'label', 'subject'])
Label distribution in S2: 0    57
1    16
Name: count, dtype: int64


In [ ]:
all_rows = []
for sid in subjects:
    print("Processing", sid)
    try:
        rows = build_subject_windows(sid)
        all_rows.extend(rows)
    except Exception as e:
        print("  Error with", sid, ":", e)

df = pd.DataFrame(all_rows)
print("Dataset shape:", df.shape)
print(df["label"].value_counts())
df.head()


Processing S10
Processing S11
Processing S13
Processing S14
Processing S15
Processing S16
Processing S17
Processing S2
Processing S3
Processing S4
Processing S5
Processing S6
Processing S7
Processing S8
Processing S9
Dataset shape: (1128, 12)
label
0    878
1    250
Name: count, dtype: int64


,eda_mean,eda_std,eda_min,eda_max,bvp_mean,bvp_std,temp_mean,temp_std,acc_mag_mean,acc_mag_std,label,subject
0,0.392279,0.040467,0.353052,0.654937,5.958031,41.860638,33.294719,0.043710,62.998618,0.266568,0,S10
1,0.382285,0.012288,0.368402,0.431082,13.783500,26.804324,33.328375,0.010541,62.909628,0.286011,0,S10
2,0.378516,0.003563,0.369681,0.401661,-3.738438,15.722678,33.326500,0.011737,63.136585,2.641558,0,S10
3,0.378188,0.004120,0.369681,0.417011,-2.940219,16.047939,33.323000,0.011554,63.607738,3.772762,0,S10
4,0.384688,0.008819,0.369681,0.417011,-0.246844,14.117096,33.343062,0.025556,63.483750,3.667550,0,S10


In [ ]:
feature_cols = [c for c in df.columns if c not in ["label", "subject"]]

X = df[feature_cols].values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

clf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.95      0.95      0.95       176
           1       0.84      0.82      0.83        50

    accuracy                           0.92       226
   macro avg       0.89      0.89      0.89       226
weighted avg       0.92      0.92      0.92       226

Confusion matrix:
 [[168   8]
 [  9  41]]


In [ ]:
import joblib

MODEL_PATH = "/content/drive/MyDrive/cognivox_wesad_rf.joblib"

joblib.dump(
    {
        "model": clf,
        "feature_cols": feature_cols,
        "window_sec": WINDOW_SEC,
        "step_sec": STEP_SEC,
        "fs_wrist": FS_WRIST,
    },
    MODEL_PATH,
)

print("Saved model to:", MODEL_PATH)


Saved model to: /content/drive/MyDrive/cognivox_wesad_rf.joblib


In [ ]:
import joblib
obj = joblib.load(MODEL_PATH)
rf = obj["model"]
feat_cols = obj["feature_cols"]

# Take one random row from df
sample_row = df.sample(1, random_state=0)
sample_X = sample_row[feat_cols].values
true_label = int(sample_row["label"].iloc[0])

pred_label = int(rf.predict(sample_X)[0])
print("True label:", true_label, "Predicted:", pred_label)


True label: 0 Predicted: 0


In [ ]:
import joblib
import numpy as np

# Load model we saved earlier (if not already in memory)
MODEL_PATH = "/content/drive/MyDrive/cognivox_wesad_rf.joblib"
obj = joblib.load(MODEL_PATH)

model = obj["model"]
feature_cols = obj["feature_cols"]

def predict_stress_from_features(feat_dict, threshold=0.6):
    """
    feat_dict: dict of features, e.g. {
        'eda_mean': ..., 'eda_std': ..., ... 'acc_mag_std': ...
    }
    threshold: probability threshold to classify as stress
    returns: label (0/1), stress_score (0-1)
    """
    x = np.array([[feat_dict[col] for col in feature_cols]])
    proba = model.predict_proba(x)[0, 1]  # probability of class 1 (stress)
    label = 1 if proba >= threshold else 0
    return label, float(proba)


def generate_suggestion(label, stress_score):
    """
    Map prediction -> human-friendly suggestion.
    """
    if label == 0:
        # non-stress
        if stress_score < 0.3:
            return "You are calm. Keep going 👍"
        else:
            return "Slight stress detected. Maintain your pace and breathing."
    else:
        # stress
        if stress_score < 0.75:
            return "You seem stressed. Try slowing your speech and taking a deep breath."
        else:
            return "High stress detected. Pause for a moment, breathe slowly, then continue."

# ---- Test on a random window from your dataset ----
sample_row = df.sample(1, random_state=1)
feat_dict = sample_row[feature_cols].iloc[0].to_dict()
true_label = int(sample_row["label"].iloc[0])

pred_label, score = predict_stress_from_features(feat_dict)
suggestion = generate_suggestion(pred_label, score)

print("True label       :", true_label)
print("Predicted label  :", pred_label)
print("Stress score     :", round(score, 3))
print("Suggestion       :", suggestion)


True label       : 0
Predicted label  : 0
Stress score     : 0.015
Suggestion       : You are calm. Keep going 👍


In [ ]:
# Choose a subject to simulate live streaming
sim_subject_id = "S2"
sub_data = load_subject(sim_subject_id)
eda, bvp, acc, temp, labels = get_wrist_signals_and_labels(sub_data)

n = len(eda)

print("Simulating real-time windows...")
for start in range(0, n - WINDOW, STEP):
    end = start + WINDOW

    # Extract features for this "live" window
    feats = extract_window_features(
        eda[start:end],
        bvp[start:end],
        acc[start:end],
        temp[start:end],
    )

    # Predict stress and suggestion
    label, score = predict_stress_from_features(feats)
    suggestion = generate_suggestion(label, score)

    # For demo: get majority true label for reference
    lab_window = labels[start:end]
    vals, counts = np.unique(lab_window, return_counts=True)
    true_lab_multi = vals[np.argmax(counts)]
    true_lab_bin = map_label_multiclass_to_binary(true_lab_multi)

    print(f"Window {start:6d}-{end:6d} | True={true_lab_bin} Pred={label} Score={score:.2f} | {suggestion}")


Simulating real-time windows...
Window      0-   320 | True=-1 Pred=0 Score=0.04 | You are calm. Keep going 👍
Window    160-   480 | True=-1 Pred=0 Score=0.03 | You are calm. Keep going 👍
Window    320-   640 | True=-1 Pred=0 Score=0.06 | You are calm. Keep going 👍
Window    480-   800 | True=-1 Pred=0 Score=0.03 | You are calm. Keep going 👍
Window    640-   960 | True=-1 Pred=0 Score=0.06 | You are calm. Keep going 👍
Window    800-  1120 | True=-1 Pred=0 Score=0.06 | You are calm. Keep going 👍
Window    960-  1280 | True=-1 Pred=0 Score=0.04 | You are calm. Keep going 👍
Window   1120-  1440 | True=0 Pred=0 Score=0.01 | You are calm. Keep going 👍
Window   1280-  1600 | True=0 Pred=0 Score=0.03 | You are calm. Keep going 👍
Window   1440-  1760 | True=0 Pred=0 Score=0.02 | You are calm. Keep going 👍
Window   1600-  1920 | True=0 Pred=0 Score=0.03 | You are calm. Keep going 👍
Window   1760-  2080 | True=0 Pred=0 Score=0.00 | You are calm. Keep going 👍
Window   1920-  2240 | True=0 Pred=0 